In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import string

# import GloVe - for word embedding
from torchtext.vocab import GloVe
global_vectors = GloVe(name='840B', dim=300)

# tokenizer
from torchtext.data import get_tokenizer

# stop word removal
nltk.download('stopwords')
from nltk.corpus import stopwords

# lemmazation
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

# models
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

# tf-idf and svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

In [ ]:
df = pd.read_csv('train.txt', delimiter='\t', header=None, names=["Label", "Review"])
df_test = pd.read_csv('test_just_reviews.txt', delimiter='\t', header=None, names=["Review"])

#separating the labels into real/fake and positive/negative
df['Realness'] = df['Label'].str[:-8]
df['Sentiment'] = df['Label'].str[-8:]
df = df.drop('Label', axis=1)

# remove capitalized letters
#df['Review'] = df['Review'].str.lower()
#df['Review'] = tokenizer(df['Review'])
data = df.to_numpy()
Xt = df_test.to_numpy()[:,0]

X = data[:,0]
yr = data[:,1]
ys = data[:,2]
print(df)

In [1]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Input text
text = ["This is an example sentence for word embedding.","Another one"]

# Tokenize the text
tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
print(f"tokens-->{tokens}")
# Get the model embeddings
output = model(**tokens)
print(f"output-->{output}")
word_embeddings = output.last_hidden_state  # The embeddings for each token

print(f"word_embeddings-->{word_embeddings}")
print(f"word_embeddings.shape-->{word_embeddings.shape}")
# Now you can use word_embeddings for further analysis or downstream tasks.

/Users/nunomachado/opt/anaconda3/envs/LN23/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tokens-->{'input_ids': tensor([[ 101, 2023, 2003, 2019, 2742, 6251, 2005, 2773, 7861, 8270, 4667, 1012,
          102],
        [ 101, 2178, 2028,  102,    0,    0,    0,    0,    0,    0,    0,    0,
            0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
output-->BaseModelOutput(last_hidden_state=tensor([[[-0.2221, -0.2035, -0.1375,  ..., -0.1363, -0.0892,  0.5583],
         [-0.6357, -0.3926, -0.4852,  ..., -0.1972,  0.2487,  0.4662],
         [-0.5783, -0.2248, -0.1971,  ..., -0.1546, -0.2780,  1.0741],
         ...,
         [ 0.0465,  0.0735, -0.1713,  ..., -0.2696, -0.3910,  0.4997],
         [ 0.8832,  0.0211, -0.6477,  ...,  0.3058, -0.8007, -0.2468],
         [ 0.6095,  0.1018, -0.2602,  ...,  0.1925, -1.0280,  0.1140]],

        [[-0.2056, -0.2218, -0.0211,  ...,  0.0203,  0.2521,  0.1857],
         [ 0.1032, -0.4571,  0.0073,  ...,  0.2376,  0.7188, -0.2225],
         [-0.4658, -0.5940, -0.3537,

In [7]:
from transformers import DistilBertTokenizer

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Input text
text = "This is an example sentence for word embedding."

# Tokenize the text
tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

# Get the token IDs
input_ids = tokens['input_ids']

# Convert token IDs to tokens
# Use .tolist() to convert the PyTorch tensor to a Python list
input_tokens = [tokenizer.convert_ids_to_tokens(ids) for ids in input_ids[0].tolist()]

# Print the tokens
print(input_tokens)

['[CLS]', 'this', 'is', 'an', 'example', 'sentence', 'for', 'word', 'em', '##bed', '##ding', '.', '[SEP]']


In [ ]:
from nlppreprocess import NLP
import pandas as pd

nlp = NLP()
df = pd.read_csv('some_file.csv')
df['text'] = df['text'].apply(nlp.process)